In [60]:
import numpy as np
import numpy.matlib
from numpy.matlib import repmat
import math as m
import os
from scipy.linalg import toeplitz
from scipy.special import gamma
from scipy.sparse import diags
import scipy.sparse
import scipy
from YC_fcbt import *
from dif1D import *

In [127]:
# physical parameters
sigma = 100;       # surface tension [mN/m]
grav = 9.807e3;    # gravitational acceleration [mm/s^2]
rneedle = 1;       # radius of the needle [mm]
volume0 = 32;      # prescribed volume in mm^3
deltarho = 1e-3;   # density difference [10^6 kg/m^3]
pi=m.pi

# numerical parameters
N = 40;          # resolution of the discretization for calculation
Nplot = 80;      # resolution of the discretization for plotting
Ncheb = 10;      # number of Chebyshev to describe the shape
alpha = 0.1;     # relaxation parameter in the Newton-Raphson scheme

# calculate the dimensionless quantities
sigmaprime = sigma/(deltarho*grav*rneedle**2)
volume0prime = volume0/rneedle**3

# predict the maximum length of the interface (empirical Nagel)
smax = m.sqrt(sigmaprime)*2.0/0.8701
# get the differentation/integration matrices and the grid
D,_,w,s = dif1D('cheb',0,smax,N,5)
# predict the shape of the interface (empirical Nagel)
z = -4/3*smax/pi*(np.cos(pi*3/4*s/smax))
z = z - max(z)
r = 4/3*smax/pi*(np.sin(pi*3/4*s/smax))
psi = pi*3/4*s/smax
C = 1; # initial stretch parameter
p0 = m.sqrt(sigmaprime)*1.5; # predict the pressure (empirical Nagel)

In [128]:
Z = np.zeros((N,N));            # matrix filled with zeros
IDL = np.hstack((1, np.zeros((N-1)))); # line with single one and rest zeros
ZL = np.zeros(N);       # line completely filled with zeros
u = np.ones((3*N+2,1)); 
b = np.ones((3*N+2,1)); # solution vector and right hand side
iter = 0; crash = 0; 

def rms(y) :
    rms = np.sqrt(np.mean(y**2))
    return rms


In [50]:
while rms(u) > 1e-10:
    iter = iter + 1

    if iter > 1200 :
        print('iter > 1200!')
        crash = 1; break
    # determine r from psi
    A11 = C*D; 
    A13 = scipy.sparse.diag(np.sin(psi)); 
    A18 = D*r; 
    b1 = -(C*D*r-np.cos(psi))

AttributeError: module 'scipy.sparse' has no attribute 'diag'

In [133]:
# determine r from psi
A11 = C*D; 
A13 = diags(np.squeeze(np.sin(psi)))
A18 = np.dot(D,r); 
b1 = -(C*D*r-np.cos(psi))
# determine z from psi 
A22 = C*D; 
A23 = scipy.sparse.diags(np.squeeze(-np.cos(psi))); 
A28 = D*z; 
b2 = -(C*D*z-np.sin(psi))

 # determine psi from Laplace law
A31 = -sigmaprime*scipy.sparse.diags(np.squeeze(np.sin(psi)/r**2))
A32 = scipy.sparse.diags(np.squeeze(np.ones(N)))
A33 = C*sigmaprime*D + sigmaprime*scipy.sparse.diags(np.squeeze(np.cos(psi)/r))
A38 = sigmaprime*(D*psi)
A39 = -np.ones(N)
b3 = p0-z-sigmaprime*(C*D*psi+np.sin(psi)/r)

  # impose the needle radius as a BC (imposes the domain length)
  # NOTE: the lengths are scaled with the radius, thus its value is one

A81 = np.flip(IDL); 
b8 = (1-r[-1])
  
  # determine pressure - use volume
A91 = 2*w*r.T*np.sin(psi.T)
A93 = w*r.T**2*np.cos(psi.T)
A98 = -volume0prime/m.pi
b9 = -(w*(r**2*np.sin(psi))-C*volume0prime/m.pi)

# boundary condition r(0) = 0
A11[0,:] = IDL; 
A13=A13.tolil()
A13[0,:] = ZL; 
A18[0] = 0
b1[0] = -r[0]
  
# boundary condition z(s0) = 0
A22[0,:] = np.flip(IDL); 
A23=A23.tolil()
A23[0,:] = ZL; 
A28[0] = 0
b2[0] = -z[-1]
  
# boundary condition phi(0) = 0
A31=A31.tolil()
A31[0,:] = ZL; 
A32=A32.tolil()
A32[0,:] = ZL; 
A33[0,:] = IDL; 
A38[0,:] = 0; 
A39[0] = 0
b3[0] = -psi[0]

# assemble matrices
Z1 = np.zeros(N).reshape(N,1)
A18=A18.reshape(N,1); 
print(A11.shape,Z.shape,A13.shape,A18.shape,Z1.shape)


np.concatenate((A11, Z, A13, A18, Z1),axis=1)
A = np.vstack(np.hstack((A11, Z, A13, A18, Z1)),
              [Z, A22, A23, A28, Z1],
              [A31, A32, A33, A38, A39],
              [A81, np.zeros(1,2*N), -1,0],
              [A91, Z1.T,A93,A98,0])
     
b = [b1,b2,b3,b8,b9]; 

# solve the system of equations
u = np.linalg.lstsq(A,b)

# update variables
r   = r   + alpha*u[0:N-1]
z   = z   + alpha*u[N:2*N-1]; 
psi = psi + alpha*u[2*N:3*N-1]; 
C   = C   + alpha*u[3*N]; 
p0  = p0  + alpha*u[3*N+1]; 


(40, 40) (40, 40) (40, 40) (40, 1) (40, 1)


C:\Users\臧义程\AppData\Local\Temp\ipykernel_24536\2809259078.py:13: RuntimeWarning: invalid value encountered in divide
  A31 = -sigmaprime*scipy.sparse.diags(np.squeeze(np.sin(psi)/r**2))
C:\Users\臧义程\AppData\Local\Temp\ipykernel_24536\2809259078.py:15: RuntimeWarning: divide by zero encountered in divide
  A33 = C*sigmaprime*D + sigmaprime*scipy.sparse.diags(np.squeeze(np.cos(psi)/r))
C:\Users\臧义程\AppData\Local\Temp\ipykernel_24536\2809259078.py:18: RuntimeWarning: invalid value encountered in divide
  b3 = p0-z-sigmaprime*(C*D*psi+np.sin(psi)/r)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 2 has 0 dimension(s)

In [100]:
np.hstack((A11, Z, A13, A18, Z1))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [33]:
np.zeros(())

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [86]:
f=0
N=len(y)
pi=m.pi
y=r
max_order=10

smax = m.sqrt(sigmaprime)*2.0/0.8701

# get the differentation/integration matrices and the grid
D,_,w,s = dif1D('cheb',0,smax,N,5)
# predict the shape of the interface (empirical Nagel)
z = -4/3*smax/m.pi*(m.cos(m.pi*3/4*s/smax))
z = z - max(z)
r = 4/3*smax/m.pi*(m.sin(m.pi*3/4*s/smax))

YC=fcbt('cheb',N,abs(max_order)+1)
if np.size(y,1)==1:
    y=y.T
ys0=y
if max_order == None:
    max_order=N/2
if shift_order == None:
    shift_order = 0
revarg = (max_order<0)
max_order=abs(max_order)
ds=2/N-1
d,_,w,s=dif1D('fd',-1+ds,2-2*ds,N-2,5)
d[0,:]=np.hstack((1,np.zeros(N-3)))
w2= np.linalg.inv(d)
w2=w2[-1,:]
w2[0]=0
c=np.zeros((max_order+1,1))
# 1st pass over 6 modes, c are the modes, from y the fitted part is subtracted
for k in range (0,min(6,max_order)+1):
    y0 = YC [0,k]*y[0]
    dy=(YC[1,k]*y(1)-YC[0,k]*y[0])/ds
    singular_left = (y0+dy)*np.acos(1-ds)-dy*np.sqrt(ds*(2-ds))
    y0 = YC[-1,k]*y[-1]
    dy = (YC[-1,k]*y[-1]-YC[-2,k]*y[-2])/ds
    singular_right = (y0-dy)*np.acos(1-ds)+dy*np.sqrt(ds*(2-ds))
    c[k] = 2/pi*(w2*(y[1:-2]/np.sqrt(1-s**2)*YC[1:-2,k]) + singular_left + singular_right)/(1+(k==0))
    y = y - c[k]*YC[:,k]



NameError: name 'y' is not defined

3.141592653589793